In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import keras

from keras.datasets import cifar10

from art.attacks import FastGradientMethod, ProjectedGradientDescent
from art.classifiers import KerasClassifier
from art.utils import load_dataset
from art.attacks import BasicIterativeMethod
from art.defences import AdversarialTrainer

from keras.datasets import cifar10
from keras.optimizers import adam
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.models import Sequential,Model,load_model
from keras import applications
from keras.optimizers import SGD, Adam
import numpy as np

import tensorflow as tf

import matplotlib.pyplot as plt

# print(keras.__version__)

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('cifar10'))
# (x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data() # difference is only one hot y vector?

### Parameters

In [3]:
USE_PARTIAL_DATA = True   # how much of data we use for training
PARTIAL_SIZE = 256*4

USE_WEIGHT_DECAY = False
USE_DATA_AUGMENTATION = False

USE_RETRAINING = True   # Either use retrain or train directly using PGD

VERBOSITY = 1  # Keras fit verbosity

In [4]:
epochs = 2
batch_size = 256
momentum = 0.9
weight_decay = 5e-4

retraining_rounds = 2
epochs_per_round = 2

lr = 0.0001 # lr = 0.01
optimizer = Adam(lr=lr) # keras.optimizers.SGD(lr=lr, momentum=momentum) # Adam(lr=lr)
loss = 'categorical_crossentropy'
metrics = ['accuracy']

#Adv training
norm = 2
eps = 0.5
steps = 7
step_size = 0.1
adv_ratio = 1.0
targeted = False

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
input_shape = x_train.shape[1:]
no_classes = y_train.shape[1]

In [6]:
model = applications.resnet50.ResNet50(weights=None, include_top=True, input_shape=input_shape, classes=no_classes)

In [7]:
if USE_WEIGHT_DECAY:   # https://jricheimer.github.io/keras/2019/02/06/keras-hack-1/
    for layer in model.layers:
        if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
            layer.add_loss(keras.regularizers.l2(weight_decay)(layer.kernel))
        if hasattr(layer, 'bias_regularizer') and layer.use_bias:
            layer.add_loss(keras.regularizers.l2(weight_decay)(layer.bias))

In [8]:
if USE_PARTIAL_DATA:
    x_train, y_train = x_train[:PARTIAL_SIZE], y_train[:PARTIAL_SIZE]
    x_test, y_test = x_test[:PARTIAL_SIZE], y_test[:PARTIAL_SIZE]

In [9]:
# model.summary()

In [10]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
classifier = KerasClassifier(model=model, clip_values=(min_, max_))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.


In [11]:
attack = ProjectedGradientDescent(classifier=classifier, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)

In [12]:
test_size = 100
print("BEFORE TRAINING:")
predictions = classifier.predict(x_test[:test_size])
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

x_test_adv = attack.generate(x=x_test[:test_size])
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

BEFORE TRAINING:
Accuracy on benign test examples: 12.0%
Accuracy on adversarial test examples: 10.0%


In [13]:
if USE_RETRAINING:
    # Initial training
    classifier.fit(x_train, y_train, nb_epochs=epochs, batch_size=batch_size, verbose=VERBOSITY)
    
    # Retraining for adversarial examples
    for i in range(retraining_rounds):
        print("Retraining round {}/{}".format(i+1, retraining_rounds))
        # Generating adv examples
        attack = ProjectedGradientDescent(classifier=classifier, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)
        adv_x_train = attack.generate(x=x_train, y=y_train)
        
        # Retraining, a few fewer epochs I guess?
        classifier.fit(adv_x_train, y_train, nb_epochs=epochs_per_round, batch_size=batch_size, verbose=VERBOSITY)

else:
    trainer = AdversarialTrainer(classifier, attack, ratio=adv_ratio)
    trainer.fit(x_train, y=y_train, nb_epochs=epochs, batch_size=batch_size, verbose=VERBOSITY)

Epoch 1/2
4/4 [==============================] - ETA: 1:34 - loss: 4.3182 - acc: 0.097 - ETA: 49s - loss: 4.0121 - acc: 0.103 - ETA: 22s - loss: 3.8129 - acc: 0.11 - 87s 22s/step - loss: 3.6492 - acc: 0.1250
Epoch 2/2
4/4 [==============================] - ETA: 55s - loss: 2.8626 - acc: 0.17 - ETA: 36s - loss: 2.7092 - acc: 0.20 - ETA: 18s - loss: 2.6393 - acc: 0.20 - 74s 18s/step - loss: 2.6811 - acc: 0.1982
Epoch 1/2
4/4 [==============================] - ETA: 49s - loss: 3.7577 - acc: 0.10 - ETA: 33s - loss: 3.6813 - acc: 0.08 - ETA: 16s - loss: 3.7408 - acc: 0.07 - 67s 17s/step - loss: 3.7406 - acc: 0.0703
Epoch 2/2
4/4 [==============================] - ETA: 49s - loss: 3.6734 - acc: 0.08 - ETA: 32s - loss: 3.3959 - acc: 0.08 - ETA: 16s - loss: 3.3061 - acc: 0.09 - 66s 17s/step - loss: 3.2289 - acc: 0.0996
Epoch 1/2
4/4 [==============================] - ETA: 47s - loss: 3.8583 - acc: 0.05 - ETA: 31s - loss: 3.7596 - acc: 0.07 - ETA: 15s - loss: 3.6864 - acc: 0.06 - 64s 16s/step -

In [14]:
print("AFTER TRAINING:")
predictions = classifier.predict(x_test[:test_size])
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

attack = ProjectedGradientDescent(classifier=classifier, norm=norm, eps=eps, eps_step=step_size, max_iter=steps, targeted=targeted, batch_size=batch_size)
x_test_adv = attack.generate(x=x_test[:test_size])
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:test_size], axis=1)) / len(y_test[:test_size])
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

AFTER TRAINING:
Accuracy on benign test examples: 19.0%
Accuracy on adversarial test examples: 12.0%


In [15]:
# Saving trained model
path = os.getcwd()
print(path)
classifier.save(filename="robust_test.h5", path=path)

C:\Users\lgran\Documents\GitHub\AdvancedDeepLearning


In [16]:
# classifier.fit(x_train, y_train, nb_epochs=25, batch_size=50, verbose=1)

In [17]:
# preds = np.argmax(classifier.predict(x_train), axis=1)
# print(preds[0])
# acc = np.sum(preds == np.argmax(y_train)) / y_train.shape[0]
# print(acc)

In [18]:
# predictions = classifier.predict(x_test)
# accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
# print('Accuracy on benign test examples: {}%'.format(accuracy * 100))
# 
# # Step 6: Generate adversarial test examples
# attack = FastGradientMethod(classifier=classifier, eps=0.2)
# x_test_adv = attack.generate(x=x_test)
# 
# # Step 7: Evaluate the ART classifier on adversarial test examples
# 
# predictions = classifier.predict(x_test_adv)
# accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
# print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

### Training with adversarial examples

In [19]:
# x_train_adv = attack.generate(x=x_train)

In [20]:
# classifier.fit(x_train_adv, y_train, nb_epochs=10, batch_size=50, verbose=1)

In [21]:
# # Step 7: Evaluate the ART classifier on adversarial test examples
# predictions = classifier.predict(x_test)
# accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
# print('Accuracy on benign test examples: {}%'.format(accuracy * 100))
# 
# predictions = classifier.predict(x_test_adv)
# accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
# print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

In [22]:
# following this tutorial https://github.com/IBM/adversarial-robustness-toolbox/blob/master/examples/adversarial_training_cifar10.py

In [23]:
# import os
# path = os.getcwd()
# print(path)
# classifier.save(filename="testing", path=path)
# 
# from keras.models import load_model
# model2 = load_model(path+"/testing")
# classifier2 = KerasClassifier(model=model2, clip_values=(min_, max_))
# 
# preds = np.argmax(classifier2.predict(x_train), axis=1)
# print(preds[0])
# acc = np.sum(preds == np.argmax(y_train)) / y_train.shape[0]
# print(acc)

In [24]:
# fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
# 
# fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
# 